In [ ]:
#Install Open AI

In [20]:
pip install OpenAi


In [47]:
from openai import OpenAI
from google.colab import userdata
import os

In [48]:
client = OpenAI(api_key="Your API OpenAI Key")

In [50]:
!pip install streamlit pyngrok openai hana-ml pandas matplotlib seaborn requests -q

In [58]:

%%writefile app.py
import streamlit as st
import json
import requests
import os
import time
import pandas as pd
from hana_ml.dataframe import ConnectionContext
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
from openai import OpenAI
import subprocess
from google.colab import userdata



HANA_ADDRESS = '434f8d46-16a2-4bb2-856d-a369e7a0b0c6.hana.trial-us10.hanacloud.ondemand.com'
HANA_PORT = 443
HANA_USER = "DBADMIN"
HANA_PASSWORD = "Your Password"
OPENAI_API_KEY = 'Your Open API Key'


hana_conn_error = None
openai_client_error = None
HANA_CONNECTION = None
client = None

## Check if SAP HANA And Open AI Connections are working fine or not
try:
    HANA_CONNECTION = ConnectionContext(address=HANA_ADDRESS,
                                        port=HANA_PORT,
                                        user=HANA_USER,
                                        password=HANA_PASSWORD)
    HANA_CONNECTION.connection.isconnected()

except Exception as e:
    hana_conn_error = e
    HANA_CONNECTION = None

try:
    client = OpenAI(api_key="Your API OpenAI Key")
    client.models.list()

except Exception as e:
    openai_client_error = e
    client = None

# --- Tool Functions ---

# Agent 1 : Query SAP HANA Database

def query_hana(sql_query: str) -> pd.DataFrame | str:
    """Run SQL on HANA and return a DataFrame or error message."""
    if HANA_CONNECTION is None:
        return f"Error: HANA Connection not established. Cannot execute query."

    st.info(f"📡 Connecting to HANA Cloud (executing query)...")
    try:
        result_df = HANA_CONNECTION.sql(sql_query).collect()
        if not isinstance(result_df, pd.DataFrame):
             st.error("❌ Error: HANA query did not return a Pandas DataFrame.")
             return "Error: Query did not return expected data format."

        st.success(f"✅ Query executed successfully. Rows returned: {len(result_df)}")
        return result_df

    except Exception as e:
        st.error(f"❌ HANA Query Failed: {e}")
        error_message = f"Error executing HANA query: {str(e)}"

        if 'inconsistent datatype' in str(e).lower() and ('sum' in str(e).lower() or 'avg' in str(e).lower()):
            error_message += "\n   Hint: This might be due to applying SUM/AVG to a non-numeric column (like 'Amount' if it's text). Consider using CAST(\"Amount\" AS DECIMAL) in the SQL."
        return error_message

# Agent 2 : Create chart out of SQL generated by Agent 1

def create_chart(data: pd.DataFrame, chart_params: dict) -> str:
    """Creates a chart from a DataFrame based on provided parameters.
       Attempts to auto-detect and apply 'hue' for comparisons if missed by AI."""

    required_params = ['chart_type', 'x_col', 'y_col', 'title']
    chart_type = chart_params.get('chart_type')
    x_col = chart_params.get('x_col')
    y_col = chart_params.get('y_col')
    title = chart_params.get('title')
    hue_col = chart_params.get('hue_col', None)


    if not all([chart_type, x_col, y_col, title]):
        missing = [k for k in required_params if not chart_params.get(k)]
        return f"Error: Missing params: {', '.join(missing)}"


    if not isinstance(data, pd.DataFrame) or data.empty: return "Error: No data for chart."
    if x_col not in data.columns: return f"Error: X-col '{x_col}' missing. Cols: {list(data.columns)}"
    if y_col not in data.columns: return f"Error: Y-col '{y_col}' missing. Cols: {list(data.columns)}"
    if hue_col and hue_col not in data.columns: return f"Error: Hue-col '{hue_col}' missing. Cols: {list(data.columns)}"



    if hue_col is None and chart_type.lower() in ['line', 'bar'] and len(data.columns) > 2:

        potential_hue_cols = []
        for col in data.columns:

            if col != x_col and col != y_col:
                 dtype_str = str(data[col].dtype).lower()
                 if 'object' in dtype_str or 'category' in dtype_str or 'string' in dtype_str:
                     unique_count = data[col].nunique()
                     if 1 < unique_count <= 10:
                         potential_hue_cols.append(col)


        best_potential_hue = None
        if potential_hue_cols:

            preferred_cols = ["Product", "Country", "Sales Person"]
            for pref_col in preferred_cols:
                if pref_col in potential_hue_cols: best_potential_hue = pref_col; break
            if not best_potential_hue: best_potential_hue = potential_hue_cols[0]


        if best_potential_hue:
            hue_col = best_potential_hue
            st.warning(f"🤖 Auto-applied category distinction using column: '{hue_col}'", icon="⚠️")

    try:
        data_copy = data.copy()
        if data_copy[y_col].dtype == 'object':

            data_copy[y_col] = pd.to_numeric(data_copy[y_col].astype(str).str.replace(',', '', regex=False), errors='coerce')
            if data_copy[y_col].isnull().any():
                st.warning(f"⚠️ Some values in '{y_col}' ignored (non-numeric).")
                data_copy.dropna(subset=[y_col], inplace=True)
                if data_copy.empty: return f"Error: No valid numeric data in '{y_col}'."
        data = data_copy

    except Exception as convert_e: st.warning(f"⚠️ Y-col conversion issue: {convert_e}")

## Data Plotting
    fig, ax = plt.subplots(figsize=(10, 5.5))
    try:
        plt.style.use('seaborn-v0_8-darkgrid')
        ax.set_facecolor('#2B3F55'); fig.patch.set_facecolor('#1E2A3A')
        ax.tick_params(colors='white'); ax.xaxis.label.set_color('white')
        ax.yaxis.label.set_color('white'); ax.title.set_color('white')

        plot_params = {'data': data, 'x': x_col, 'y': y_col, 'ax': ax}
        if hue_col:
            plot_params['hue'] = hue_col
            if chart_type.lower() == 'bar': plot_params['dodge'] = True
            if chart_type.lower() == 'line': plot_params['marker'] = 'o'



        if chart_type.lower() == 'bar': sns.barplot(**plot_params)
        elif chart_type.lower() == 'line': sns.lineplot(**plot_params)
        else: plt.close(fig); return f"Error: Unsupported chart: '{chart_type}'."

        ax.set_title(title, fontsize=14, fontweight='bold')
        ax.set_xlabel(x_col, fontsize=12)
        ax.set_ylabel(y_col, fontsize=12)
        if hue_col:
            legend = ax.legend(title=hue_col, bbox_to_anchor=(1.05, 1), loc='upper left', facecolor='#2B3F55', edgecolor='white')
            plt.setp(legend.get_texts(), color='white')
            plt.setp(legend.get_title(), color='white', fontweight='bold')
        if data[x_col].nunique() > 8 and data[x_col].dtype == 'object': plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
        else: plt.setp(ax.get_xticklabels(), rotation=0)
        plt.tight_layout(rect=[0, 0, 0.85, 1])
        safe_title = "".join(c if c.isalnum() else "_" for c in title)
        if not os.path.exists("charts"): os.makedirs("charts")
        chart_filename = f"charts/chart_{safe_title}_{int(time.time())}.png"
        plt.savefig(chart_filename, bbox_inches='tight', facecolor=fig.get_facecolor(), transparent=False)
        plt.close(fig)
        st.success(f"✅ Chart saved as {chart_filename}")

        return f"Chart '{title}' created and saved as {chart_filename}"

    except Exception as e:
        st.error(f"❌ Error creating chart: {e}")

        st.error(f"Traceback:\n{traceback.format_exc()}")
        plt.close(fig)

        return f"Error creating chart: {e}"

# Agent 2 : To call Weather API real time
def get_weather(city:str):
    """Gets the weather for a given city."""
    st.write(f"⛏️ Tool Called: get_weather, City: {city}")
    url = f"https://wttr.in/{city}?format=%C+%t"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        st.success(f"✅ Weather data retrieved for {city}")
        return f"The weather in {city} is {response.text}"
    except requests.exceptions.RequestException as e:
        st.error(f"❌ Error fetching weather for {city}: {e}")
        return f"Error fetching weather for {city}: {e}"

# Agent 3 : To run command in your  current directory
def run_command(command: str) -> str:
    """Executes a shell command."""
    st.write(f"⛏️ Tool Called: run_command, Command: {command}")
    allowed_commands = ['ls', 'pwd', 'echo']
    try:
        command_parts = command.strip().split()
        if not command_parts:
             return "Error: Empty command provided."
        command_base = command_parts[0]
    except IndexError:
        return "Error: Empty command provided."

    if command_base not in allowed_commands:
         st.warning(f"❌ Blocked unsafe command: {command}")
         return f"Error: Command '{command_base}' is not allowed."
    try:
        import subprocess
        process = subprocess.run(command, shell=True, capture_output=True, text=True, timeout=15, check=False)
        output = process.stdout + process.stderr
        if process.returncode == 0:
            st.success(f"✅ Command executed successfully.")
            return f"Command output:\n'''\n{output.strip()}\n'''"
        else:
            st.error(f"❌ Command failed with return code {process.returncode}")
            return f"Command failed (code {process.returncode}):\n'''\n{output.strip()}\n'''"
    except Exception as e:
        st.error(f"❌ Error executing command '{command}': {e}")
        return f"Error executing command: {e}"

## Available Tools

available_tools = {
    "get_weather": {
        "fn": get_weather,
        "description": "Takes a city name and returns the current weather for that city."
    },
    "run_command": {
        "fn": run_command,
        "description": "Takes a *safe* shell command string (like 'ls', 'pwd') and executes it, returning the output. Use with caution."
    },
    "query_hana": {
        "fn": query_hana,
        "description": "Takes a valid SQL query string for the CHOCOLATE_SALES table. Executes the query against HANA Cloud. Returns the resulting data (as a DataFrame) on success, which is made available for subsequent steps (like charting), or an error message string on failure."
    },
    "create_chart": {
         "fn": create_chart,
         "description": "Takes a JSON object input with keys 'chart_type' (string: 'bar' or 'line'), 'x_col' (string: column name for X-axis), 'y_col' (string: column name for Y-axis), and 'title' (string). It uses the data obtained from the *immediately preceding* successful 'query_hana' call to generate and save a chart image. Returns a confirmation message including the filename or an error."
     }
}


system_prompt = """
You are a helpful AI Assistant specialized in resolving user queries using available tools.
You operate in four charming steps: Start → Plan → Action → Observe.

Your mission: For any user query and available tools, break the problem down into a plan,
select the appropriate tool based on your brilliant deduction, call that tool, wait for its result,
and finally resolve the user’s query.

---

Rules:
1. Always follow the strict JSON output format — but make it delightful!
{
  "step": "plan|action|output",
  "content": "Your thought process for plan/output, or empty for action.",
  "function": "tool_name (only if step is action)",
  "input": "tool_input (string for most, JSON object for create_chart - only if step is action)"
}
2. Perform only one step at a time, like a graceful ballet.
3. Carefully analyze the user query using your sharp wit.
4. When using the 'query_hana' tool, generate **SQL for the CHOCOLATE_SALES table** using proper rules and double-quote column names.
5. When the user asks for a chart or visualization:
   a. First, use 'query_hana' to fetch the necessary aggregated or filtered data. Ensure your SQL query selects the exact columns needed for the chart axes (e.g., month, sales amount).
   b. **Crucially, if the request involves comparing multiple categories (like different products, countries, or sales persons) on the *same* chart, your SQL query *must* also select the category column itself (e.g., "Product", "Country").**
   c. Then, in the *next* action step, use the 'create_chart' tool.
   d. For 'create_chart', provide a JSON object for the 'input' field containing: 'chart_type', 'x_col', 'y_col', and a descriptive 'title'.
   e. **MANDATORY FOR COMPARISONS:** If the query involves comparing categories (see rule 5b), you **MUST** include the 'hue_col' parameter in the 'create_chart' JSON input. Set its value to the name of the category column selected in your SQL (e.g., '"hue_col": "Product"'). **Failure to include 'hue_col' when comparing categories will result in an incorrect chart.**
6.  **IMPORTANT FOR DATA DISPLAY:** When the user asks for data directly (not a chart) and you have successfully run 'query_hana':
    a. In your final 'output' step, formulate your response conversationally.
    b. Include the exact placeholder '%%DATA_TABLE%%' where the retrieved data should be displayed by the system. **Crucial:** Your *only* way to show the data is to put the literal string '%%DATA_TABLE%%' in your final 'output' step's content. The system will replace this. **Do not** try to list the data yourself or summarize it beyond mentioning the count if relevant. Just use the placeholder.

7.**AVOID FULL TABLE QUERIES:** Do **NOT** generate SQL queries that attempt to select all columns ('SELECT *') or many raw columns without any filtering ('WHERE'), aggregation ('SUM', 'COUNT', 'AVG', 'GROUP BY'), or row limitation ('LIMIT').
    Queries like 'SELECT * FROM CHOCOLATE_SALES' or 'SELECT "Sales Person", "Country", "Product", "Date", "Amount", "Boxes Shipped" FROM CHOCOLATE_SALES' are **forbidden** as they are inefficient and unhelpful.

- **CLARIFY BROAD REQUESTS:** If the user asks for "all data", "show the table", or makes a similarly broad request that implies fetching the entire table without refinement:
    - Your 'plan' step should recognize this is too broad.
    - Your next step should be 'output', not 'action:query_hana'.
    - In the 'output' step's 'content', politely explain that you cannot fetch the entire table and ask the user to be more specific. Suggest options like:
        - Applying filters (e.g., "for a specific product like 'Eclairs'?", "in a particular country like 'USA'?", "within a date range?").
        - Requesting a summary (e.g., "total sales per product?", "count of sales per country?").
        - Specifying the exact columns they need (if it's a small, targeted number).
    - **Example Clarification Output JSON:** '{"step": "output", "content": "Fetching the entire sales table isn't practical. Could you please help me narrow it down? For example, are you interested in sales for a specific product, a particular country, a date range, or perhaps a summary like total sales per product?", "function": null, "input": null}'
- **PRIORITIZE SPECIFICITY:** Always aim to generate queries that are targeted to the user's specific question using 'WHERE', 'GROUP BY', 'SUM'/'AVG'/'COUNT', and/or 'LIMIT'. Only select the specific columns needed to answer the question or create the requested chart.




---

CHOCOLATE_SALES TABLE:

Columns:
- "Sales Person" (string)
- "Country" (string)
- "Product" (string)
- "Date" (date, format YYYYMMDD)
- "Amount" (float - **potential issue: might be TEXT in DB, use CAST("Amount" AS DECIMAL(18, 2)) in SUM/AVG if query fails**)
- "Boxes Shipped" (integer)


SQL Guidelines:
- Always select from table: CHOCOLATE_SALES
- Use double quotes for all column names: e.g., "Country", "Amount"
- Use GROUP BY, ORDER BY, SUM, AVG, COUNT, LIMIT etc. as needed.
- Use aliases for aggregated columns (e.g., 'SUM("Amount") AS "Total Amount"'). Make sure these aliases match the 'x_col'/'y_col' used in 'create_chart'. Use double quotes for aliases too if they contain spaces or are reserved words.
- Use ORDER BY and LIMIT for top-N queries.
- **If SUM or AVG on "Amount" fails with a datatype error, regenerate the SQL using 'CAST("Amount" AS DECIMAL(18, 2))' inside the aggregate function.**
- **When filtering on string columns like "Product" or "Sales Person", use the 'LIKE' operator with '%' wildcards around the user's term (e.g., 'WHERE "Product" LIKE '%Mint Chip%'') instead of an exact match ('='), unless the user specifically requests an exact match (e.g., "sales person is exactly 'John Doe'"). Case sensitivity might depend on HANA settings, 'UPPER()' can be used on both sides for case-insensitive matching if needed (e.g. 'WHERE UPPER("Product") LIKE UPPER('%Mint Chip%')').**
- Output **only valid SQL**, no explanations.
- Generate only syntactically correct and optimized SQL.
- **IMPORTANT FOR STRING FILTERING:** When filtering on string columns like "Product", "Country", or "Sales Person" based on user input that might have inconsistent casing (e.g., 'eclairs' vs 'Eclairs', 'usa' vs 'USA'):
    - **ALWAYS** use the 'UPPER()' function on both the database column and the user's search term within the 'LIKE' operator.
    - **ALWAYS** wrap the user's term with ''%'' wildcards unless they explicitly ask for an exact match *and* case sensitivity (which is rare).
    - **Example:** 'WHERE UPPER("Product") LIKE UPPER('%eclairs%')'
    - **Example for specific list:** 'WHERE UPPER("Product") IN (UPPER('Mint Chip Choco'), UPPER('85% Dark Bars'))'
    - Only omit 'UPPER()' if the user *specifically requests a case-sensitive search*.

---

Available tools Descriptions:
- get_weather: Takes a city name and returns the current weather for that city.
- run_command: Takes a *safe* shell command string (like 'ls', 'pwd') and executes it, returning the output. Use with caution.
- query_hana: Takes a valid SQL query string for the CHOCOLATE_SALES table. Executes the query against HANA Cloud. Returns the resulting data (as a DataFrame) on success, or an error message string on failure. The system makes successful results available for subsequent steps.
- create_chart: Takes a JSON object input with keys 'chart_type' (string: 'bar' or 'line'), 'x_col' (string: column name for X-axis matching SQL output), 'y_col' (string: column name for Y-axis matching SQL output), and 'title' (string). Uses data from the preceding successful 'query_hana' call. Returns a confirmation message with filename or an error.

---

Example 1: Weather
User Query: What's the weather in Tokyo?
Output: {"step":"plan", "content": "The user wants the weather in Tokyo. I should use the get_weather tool.", "function": null, "input": null}
Output: {"step":"action", "content": "", "function":"get_weather", "input":"Tokyo"}
Output: {"step":"observe", "output":"The weather in Tokyo is Sunny +22°C"}
Output: {"step":"output", "content":"The current weather in Tokyo is Sunny and +22°C.", "function": null, "input": null}

---

Example 2: Top Sales People (Data Only)
User Query: Show me top 5 sales people by amount.
Output: {"step":"plan", "content":"The user wants the top 5 sales people ranked by total sales amount. I need to query the HANA database.", "function": null, "input": null}
Output: {"step":"action", "content": "", "function":"query_hana", "input":"SELECT \"Sales Person\", SUM(CAST(\"Amount\" AS DECIMAL(18,2))) AS \"Total Sales\" FROM CHOCOLATE_SALES GROUP BY \"Sales Person\" ORDER BY \"Total Sales\" DESC LIMIT 5"}
Output: {"step":"observe", "output":"HANA query executed successfully. 5 rows retrieved. Data is available for the next step."}
Output: {"step":"output", "content":"Okay, I've retrieved the top 5 sales people by total sales amount. The system will now display the data table:\n%%DATA_TABLE%%", "function": null, "input": null}
# Python code will replace %%DATA_TABLE%% with the actual data table from the query result.


---

Example 3: Charting
User Query: Create a bar chart showing total sales amount per country.
Output: {"step":"plan", "content":"User wants a bar chart of total sales by country. First, query data.", "function": null, "input": null}
Output: {"step":"action", "content": "", "function":"query_hana", "input":"SELECT \"Country\", SUM(CAST(\"Amount\" AS DECIMAL(18,2))) AS \"Total Amount\" FROM CHOCOLATE_SALES GROUP BY \"Country\" ORDER BY \"Country\""}
Output: {"step":"observe", "output":"HANA query executed successfully. Data is available for the next step."}
Output: {"step":"plan", "content":"Data fetched. Now, create the bar chart using the 'Country' and 'Total Amount' columns.", "function": null, "input": null}
Output: {"step":"action", "content": "", "function":"create_chart", "input": {"chart_type": "bar", "x_col": "Country", "y_col": "Total Amount", "title": "Total Sales Amount by Country"}}
Output: {"step":"observe", "output":"Chart 'Total Sales Amount by Country' created and saved as charts/chart_Total_Sales_Amount_by_Country_1678886400.png"}
Output: {"step":"output", "content":"Done! I've created the bar chart showing total sales amount by country. It's saved as charts/chart_Total_Sales_Amount_by_Country_1678886400.png.", "function": null, "input": null}

---

Example 4: Specific Data Request (Using %%DATA_TABLE%%)
User Query: List all sales made by 'Jehu Rudeforth' in the UK
Output: {"step": "plan", "content": "User wants to see all sales records for a specific person ('Jehu Rudeforth') in a specific country ('UK'). I need to query the CHOCOLATE_SALES table.", "function": null, "input": null}
Output: {"step": "action", "content": "", "function": "query_hana", "input": "SELECT * FROM CHOCOLATE_SALES WHERE \"Sales Person\" LIKE '%Jehu Rudeforth%' AND \"Country\" LIKE '%UK%'"}
Output: {"step": "observe", "output": "HANA query executed successfully. 12 rows retrieved. Data is available for the next step."}
Output: {"step": "output", "content": "Okay, here are the sales records for 'Jehu Rudeforth' in the UK:\n%%DATA_TABLE%%", "function": null, "input": null}

Example 5: Top and Bottom Request
User Query: Show the best and worst selling products by total amount.
Output: {"step": "plan", "content": "User wants the single product with the highest total sales and the single product with the lowest total sales. I need to use UNION ALL to combine two queries.", "function": null, "input": null}
Output: {"step": "action", "content": "", "function": "query_hana", "input": "(SELECT 'Top Selling' AS \"Rank\", \"Product\", SUM(CAST(\"Amount\" AS DECIMAL(18,2))) AS \"Total Sales\" FROM CHOCOLATE_SALES GROUP BY \"Product\" ORDER BY \"Total Sales\" DESC LIMIT 1) UNION ALL (SELECT 'Worst Selling' AS \"Rank\", \"Product\", SUM(CAST(\"Amount\" AS DECIMAL(18,2))) AS \"Total Sales\" FROM CHOCOLATE_SALES GROUP BY \"Product\" ORDER BY \"Total Sales\" ASC LIMIT 1)"}
Output: {"step": "observe", "output": "HANA query executed successfully. 2 rows retrieved. Data is available for the next step."}
Output: {"step": "output", "content": "Okay, here are the best and worst selling products by total sales amount:\n%%DATA_TABLE%%", "function": null, "input": null}


Example 6: Handling Broad Request**
User Query: Show me all the data in the chocolate sales table.
Output: {"step":"plan", "content":"The user is asking to see the entire CHOCOLATE_SALES table. This is too broad according to the guidelines. I must ask for clarification instead of querying.", "function": null, "input": null}
Output: {"step":"output", "content":"Fetching the entire sales table isn't practical as it can be very large. Could you please help me narrow down what you're looking for? For example, are you interested in sales data for a specific product, a particular country, a certain date range, or perhaps a summary like total sales per product?", "function": null, "input": null}

"""



# Start of Streamlit App UI and Logic

st.set_page_config(layout="wide")
st.title("🤖 HANA AI Assistant")


#Start of Check Connections
if hana_conn_error:
    st.sidebar.error(f"HANA Connection Failed: {hana_conn_error}")
if openai_client_error:
    st.sidebar.error(f"OpenAI Client Failed: {openai_client_error}")
if not hana_conn_error and not openai_client_error:
    st.sidebar.success("HANA & OpenAI Connections Successful!")

if HANA_CONNECTION is None or client is None:
     st.error("connection failed.")
     st.stop()
#End of Check Connections

# Initialize session state variables
if 'messages' not in st.session_state:
    st.session_state.messages = [{"role": "system", "content": system_prompt}]
if 'last_query_result_df' not in st.session_state:
    st.session_state.last_query_result_df = None
if 'processing' not in st.session_state:
    st.session_state.processing = False
if 'final_output_message' not in st.session_state:
    st.session_state.final_output_message = None
if 'status_messages' not in st.session_state:
    st.session_state.status_messages = []
if 'chart_filename' not in st.session_state:
    st.session_state.chart_filename = None


with st.form("query_form"):
    user_query = st.text_area(
        "Enter your query:",
        placeholder="e.g., Show top 5 products by sales, Create a sales chart by country, What's the weather in London?",
        height=100,
        key="user_query_input"
    )
    submit_button = st.form_submit_button("Submit Query")



st.write("Or try an example:")
cols = st.columns(3)

output_placeholder = st.empty()


# Expander Widget
def display_output():
    """Displays the final output clearly and hides details in an expander."""
    with output_placeholder.container():
        st.subheader("Assistant Response:")
        final_message_content = None
        final_message_type = None
        details_messages = []
        processed_message_ids = set()


        status_messages_copy = list(st.session_state.status_messages)


        terminating_error = None
        for i, (msg_type, msg_content) in enumerate(status_messages_copy):
            msg_id = (i, msg_type, str(msg_content)) # Creates a unique ID

            is_final_markdown = (msg_type == "markdown" and isinstance(st.session_state.final_output_message, str) and msg_content == st.session_state.final_output_message)
            is_final_type = (msg_type == "final")

            is_terminating_error = (msg_type == "error" and st.session_state.get('task_complete', False))

            if is_final_markdown or is_final_type:
                final_message_content = msg_content
                final_message_type = msg_type
                processed_message_ids.add(msg_id)

            if is_terminating_error:
                 terminating_error = msg_content
                 processed_message_ids.add(msg_id)



        if terminating_error:
             st.error(terminating_error)
        elif final_message_content:
             if final_message_type == "markdown":
                 st.markdown(final_message_content, unsafe_allow_html=True)
             else:
                 st.markdown(f"**Final Answer:**\n\n{final_message_content}")


        if st.session_state.chart_filename and os.path.exists(st.session_state.chart_filename):
             st.image(st.session_state.chart_filename)

             if final_message_content and st.session_state.chart_filename not in final_message_content:
                  st.caption(f"(Chart saved as: {st.session_state.chart_filename})")
             elif not final_message_content and not terminating_error:
                  st.success(f"Chart created and saved as {st.session_state.chart_filename}")



        for i, (msg_type, msg_content) in enumerate(status_messages_copy):
            msg_id = (i, msg_type, str(msg_content))
            is_task_complete_success = (msg_type == "success" and "Task Complete" in str(msg_content))


            if msg_id not in processed_message_ids and not is_task_complete_success:
                 details_messages.append((msg_type, msg_content))


        if details_messages:
            with st.expander("Show Execution Details"):
                for status_type, msg_content in details_messages:
                    if status_type == "info": st.info(msg_content)
                    elif status_type == "warning": st.warning(msg_content)
                    elif status_type == "error": st.error(msg_content)
                    elif status_type == "success": st.success(msg_content)
                    elif status_type == "write": st.write(msg_content)
                    elif status_type == "markdown": st.markdown(msg_content, unsafe_allow_html=True)
                    elif status_type == "dataframe":
                        if isinstance(msg_content, pd.DataFrame):
                            st.write("📊 Sample Data (first 10 rows):")
                            st.dataframe(msg_content.head(10))
                        else:
                            st.warning("Received 'dataframe' status but content is not a DataFrame.")
                    elif status_type == "dataframe_empty": st.write("Query returned no data.")


# Processing Logic

if submit_button and user_query and not st.session_state.processing:
    st.session_state.processing = True
    st.session_state.task_complete = False
    st.session_state.messages = [{"role": "system", "content": system_prompt}]
    st.session_state.messages.append({"role": "user", "content": user_query})
    st.session_state.last_query_result_df = None
    st.session_state.final_output_message = None
    st.session_state.status_messages = []
    st.session_state.chart_filename = None

    with st.spinner("🤖 Assistant thinking..."):
        max_steps = 10
        step_count = 0


        while not st.session_state.task_complete and step_count < max_steps:
            step_count += 1
            current_status = []


            try:
                #Call the OpenAI API
                response = client.chat.completions.create(
                    model='gpt-4o',
                    response_format={"type": "json_object"},
                    messages=st.session_state.messages,
                    temperature=0.1
                )
                assistant_response_content = response.choices[0].message.content
                st.session_state.messages.append({"role": "assistant", "content": assistant_response_content})

            except Exception as e:
                error_msg = f"❌ Error calling OpenAI API: {e}"
                current_status.append(("error", error_msg))
                st.session_state.task_complete = True
                st.session_state.final_output_message = error_msg
                st.session_state.status_messages.extend(current_status)
                st.session_state.processing = False
                st.rerun()

            if st.session_state.task_complete: continue

            try:

                parsed_output = json.loads(assistant_response_content)
                step = parsed_output.get("step")

                if step == "plan":
                    plan_content = parsed_output.get('content', '...')
                    current_status.append(("info", f"🧠 Plan: {plan_content}"))

                elif step == "action":
                    tool_name = parsed_output.get("function")
                    tool_input = parsed_output.get("input")
                    current_status.append(("write", f"🎬 Action: Calling tool '{tool_name}'..."))

                    if not tool_name or tool_name not in available_tools:
                        tool_output = f"Error: Tool '{tool_name}' unavailable/unspecified."
                        current_status.append(("error", f"❌ {tool_output}"))
                        st.session_state.messages.append({"role": "assistant", "content": json.dumps({"step": "observe", "output": tool_output})})
                    else:
                        tool_function = available_tools[tool_name]["fn"]
                        try:
                            tool_output = None
                            if tool_name == "query_hana":
                                sql_to_execute = str(tool_input) if isinstance(tool_input, str) else str(tool_input)
                                current_status.append(("markdown", f"**Generated SQL:**\n'''sql\n{sql_to_execute}\n'''"))
                                query_result = tool_function(sql_to_execute)
                                if isinstance(query_result, pd.DataFrame):
                                    st.session_state.last_query_result_df = query_result
                                    tool_output = f"HANA query executed. Rows: {len(query_result)}."
                                    if not query_result.empty: current_status.append(("dataframe", query_result))
                                    else: current_status.append(("dataframe_empty", None))
                                else:
                                    st.session_state.last_query_result_df = None
                                    tool_output = query_result
                                    current_status.append(("error", f"HANA Query Observation: {tool_output}"))

                            elif tool_name == "create_chart":
                                if not isinstance(tool_input, dict): tool_output = "Error: Input must be JSON."
                                elif st.session_state.last_query_result_df is None: tool_output = "Error: No data."
                                elif st.session_state.last_query_result_df.empty: tool_output = "Error: Data is empty."
                                else: tool_output = tool_function(data=st.session_state.last_query_result_df, chart_params=tool_input)

                                if isinstance(tool_output, str) and tool_output.startswith("Chart "):
                                    try:
                                        filename = tool_output.split(" saved as ")[-1]
                                        if os.path.exists(filename):
                                            st.session_state.chart_filename = filename
                                            current_status.append(("image", filename))
                                        else: current_status.append(("warning", f"Chart file not found: {filename}"))
                                    except Exception: current_status.append(("warning", "Filename parse error."))
                                else: current_status.append(("error", f"Chart Creation Observation: {tool_output}"))

                            else:
                                tool_output = tool_function(tool_input)

                                if "error" in str(tool_output).lower(): current_status.append(("warning", f"Tool Obs: {tool_output}"))
                                else: current_status.append(("write", f"Tool Obs: {tool_output}"))


                            if tool_output is None: tool_output = "Tool action complete."
                            st.session_state.messages.append({"role": "assistant", "content": json.dumps({"step": "observe", "output": str(tool_output)})})

                        except Exception as tool_e:
                            tool_output = f"Critical Error executing {tool_name}: {tool_e}"
                            current_status.append(("error", f"❌ {tool_output}"))
                            st.session_state.messages.append({"role": "assistant", "content": json.dumps({"step": "observe", "output": tool_output})})


                elif step == "output":
                    final_content = parsed_output.get('content', "...")
                    placeholder = "%%DATA_TABLE%%"
                    st.session_state.task_complete = True

                    if placeholder in final_content:
                        if st.session_state.last_query_result_df is not None and not st.session_state.last_query_result_df.empty:
                            try:
                                data_markdown = st.session_state.last_query_result_df.to_markdown(index=False)
                                formatted_content = final_content.replace(placeholder, f"\n'''markdown\n{data_markdown}\n'''\n", 1)
                            except Exception:
                                data_string = st.session_state.last_query_result_df.to_string(index=False, max_rows=20)
                                formatted_content = final_content.replace(placeholder, f"\n'''\n{data_string}\n'''\n", 1)
                            st.session_state.final_output_message = formatted_content
                            current_status.append(("markdown", formatted_content))
                        elif st.session_state.last_query_result_df is not None:
                            formatted_content = final_content.replace(placeholder, "\n(Query returned no rows.)\n", 1)
                            st.session_state.final_output_message = formatted_content
                            current_status.append(("markdown", formatted_content))
                        else:
                            formatted_content = final_content.replace(placeholder, "\n(No data available.)\n", 1)
                            st.session_state.final_output_message = formatted_content
                            current_status.append(("markdown", formatted_content))
                    else:
                        st.session_state.final_output_message = final_content
                        current_status.append(("final", final_content))


                    if st.session_state.chart_filename and isinstance(st.session_state.final_output_message, str) and st.session_state.chart_filename not in st.session_state.final_output_message:
                         st.session_state.final_output_message += f"\n(Chart: {st.session_state.chart_filename})"

                         if current_status:
                             last_type, _ = current_status[-1]
                             if last_type in ["markdown", "final"]: current_status[-1] = (last_type, st.session_state.final_output_message)
                             else: current_status.append(("final", st.session_state.final_output_message))

                    current_status.append(("success", "✅ Task Complete."))

                else:
                    error_msg = f"❓ Unknown step: '{step}'"
                    current_status.append(("error", error_msg))
                    st.session_state.messages.append({"role": "assistant", "content": json.dumps({"step": "observe", "output": error_msg})})
                    st.session_state.task_complete = True
                    st.session_state.final_output_message = error_msg

            except json.JSONDecodeError as json_e:
                error_msg = f"❌ Invalid JSON: {assistant_response_content}\nError: {json_e}"
                current_status.append(("error", error_msg))
                st.session_state.task_complete = True
                st.session_state.final_output_message = error_msg
            except Exception as proc_e:
                 error_msg = f"❌ Processing error: {proc_e}"
                 current_status.append(("error", error_msg))
                 st.session_state.task_complete = True
                 st.session_state.final_output_message = error_msg


            st.session_state.status_messages.extend(current_status)



        if not st.session_state.task_complete and step_count >= max_steps:
            st.warning("⚠️ Assistant reached maximum processing steps.")
            st.session_state.status_messages.append(("warning", "Reached max steps."))
            st.session_state.task_complete = True


    #Processing Finished
    st.session_state.processing = False

    display_output()

elif not st.session_state.processing:
    display_output()



Overwriting app.py


In [54]:
!pip install pyngrok --quiet

from pyngrok import ngrok
import os


ngrok.kill()



NGROK_AUTH_TOKEN = "Your NGROK AUTH Token"
if NGROK_AUTH_TOKEN:
  ngrok.set_auth_token(NGROK_AUTH_TOKEN)
else:
  print("Consider setting an ngrok authtoken for better stability.")

##Runs in background
!nohup streamlit run app.py --server.port 8501 --server.headless true > streamlit.log &

import time
time.sleep(5)


try:
    public_url = ngrok.connect(8501)

    print("🚀 Streamlit App is live! Click the link below:")
    print(public_url)
except Exception as e:
    print(f"❌ Error establishing ngrok tunnel: {e}")




nohup: redirecting stderr to stdout
🚀 Streamlit App is live! Click the link below:
NgrokTunnel: "https://b53f-35-245-246-49.ngrok-free.app" -> "http://localhost:8501"
